In [1]:
# https://mlflow.org/docs/latest/tracing/integrations/openai-agent
# https://openai.github.io/openai-agents-python/tracing/

In [2]:
import os 
from pydantic import BaseModel, Field
from dotenv import load_dotenv
from openai import AsyncOpenAI
from agents import (

    Agent,
    Runner,
    OpenAIChatCompletionsModel,
    ModelProvider,
    Model,
    RunConfig,
    set_default_openai_client, 
    set_default_openai_api,
    set_tracing_disabled,
    set_tracing_export_api_key,
    trace,
    

    function_tool,
)
load_dotenv()

True

In [3]:
client = AsyncOpenAI(
    base_url="https://models.inference.ai.azure.com",
    api_key=os.environ["GITHUB_TOKEN"],
)

set_default_openai_client(client)
set_default_openai_api(
    ['chat_completions']
)
set_tracing_export_api_key(os.environ["OPENAI_TRACING_KEY"])
set_tracing_disabled(False)

# tr = trace(workflow_name="msdiagentic_hello_tracking")





In [4]:
# tr.start()

In [5]:
import random

# TODO: add a strongly typed return value for the get_weather function 
# exmple: def get_weather(city: str) -> WeatherInfo:
# https://github.com/openai/openai-agents-python/blob/main/examples/basic/tools.py


class Weather(BaseModel):
    city: str
    temperature: str
    conditions: str


class CurrentTime(BaseModel):
    location: str = Field(..., description="The name of the location")
    current_time: str = Field(..., description="The current time in the location")


@function_tool
def get_weather(city: str):
    print(f"[debug] getting weather for {city}")
    temperature = f"{random.randint(-10, 50)}C"
    conditions = random.choice(["Sunny", "Windy", "Rainy", "Cloudy"])
    return Weather(city=city, temperature=temperature, conditions=conditions)  

@function_tool
def get_current_time(location):
    from datetime import datetime
    """Get the current time for a given location"""
    print(f"[debug] get_current_time called with location: {location}")
    location_lower = location.lower()


    current_time = datetime.now().strftime("%I:%M %p")
    return CurrentTime(
        location=location,
        current_time=current_time,
    )


In [6]:
class GitHubModelProvider(ModelProvider):
    def get_model(self, model_name) -> Model:
        return OpenAIChatCompletionsModel(model="gpt-4o", openai_client=client)

GITHUB_MODEL_PROVIDER = GitHubModelProvider()   

In [7]:
agent = Agent(
    name="Assistant", 
    instructions="Answer the user's questions.",
    tools=[get_weather, get_current_time],

)

In [8]:

"""This example uses a custom provider for some calls to Runner.run(), and direct calls to OpenAI for
others. Steps:
1. Create a custom OpenAI client.
2. Create a ModelProvider that uses the custom client.
3. Use the ModelProvider in calls to Runner.run(), only when we want to use the custom LLM provider.

Note that in this example, we disable tracing under the assumption that you don't have an API key
from platform.openai.com. If you do have one, you can either set the `OPENAI_API_KEY` env var
or call set_tracing_export_api_key() to set a tracing specific key.
"""

"This example uses a custom provider for some calls to Runner.run(), and direct calls to OpenAI for\nothers. Steps:\n1. Create a custom OpenAI client.\n2. Create a ModelProvider that uses the custom client.\n3. Use the ModelProvider in calls to Runner.run(), only when we want to use the custom LLM provider.\n\nNote that in this example, we disable tracing under the assumption that you don't have an API key\nfrom platform.openai.com. If you do have one, you can either set the `OPENAI_API_KEY` env var\nor call set_tracing_export_api_key() to set a tracing specific key.\n"

In [10]:
# https://github.com/openai/openai-agents-python/blob/main/examples/model_providers/custom_example_provider.py

with trace(workflow_name="msdiagentic_hello_tracking"):
    result = await Runner.run(
        agent, 
        "What is the weather in New York? And what is the time?", 
        run_config=RunConfig(model_provider=GITHUB_MODEL_PROVIDER),
        
    )

print(result.final_output)

[debug] getting weather for New York
[debug] get_current_time called with location: New York
The weather in New York is currently 5°C and cloudy. The time in New York is 01:46 AM.


In [11]:
from datetime import datetime



timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
tr = trace(
    workflow_name=f"Dummy_{timestamp}",
    group_id="Dummy Group", 
    metadata={
        "key1": "value1",
        "key2": "value2",
    },
)

# print trace workflow name
print(f"Trace workflow name: {tr.name}")
tr.start()

trace_id = tr.trace_id
print(f"Trace ID: {trace_id}")

run_config = RunConfig(
    model_provider=GITHUB_MODEL_PROVIDER,
    trace_id=trace_id,  # Pass the trace ID to the run config
    trace_metadata={"AgentName": "WeatherAgent"}, 
    workflow_name=tr.name,  # Pass the workflow name to the run config
    group_id=tr.group_id,  # Pass the group ID to the run config
)


Trace workflow name: Dummy_20250330_014641
Trace ID: trace_094ade8170f9482a92af5991fb0d9fdd


In [12]:
result = await Runner.run(
        agent, 
        "What is the weather in New York? And time.", 
        run_config=run_config,
    )

print(result.final_output)

[debug] getting weather for New York
[debug] get_current_time called with location: New York
The weather in New York is currently 14°C and rainy. The time in New York is 01:46 AM.


In [13]:
result = await Runner.run(
        agent, 
        "What is the time in Sydney", 
        run_config=run_config,
    )

print(result.final_output)

[debug] get_current_time called with location: Sydney
The current time in Sydney is 1:47 AM.


In [14]:
result = await Runner.run(
        agent, 
        "Tell me a joke!", 
        run_config=run_config,
    )

print(result.final_output)

Sure! Here's one for you:

Why don't skeletons fight each other?

Because they don't have the guts! 😄


In [15]:
tr.finish()

In [18]:
tracing_url = f"https://platform.openai.com/traces/trace?trace_id={trace_id}"

print(f"Tracing URL: {tracing_url}")

Tracing URL: https://platform.openai.com/traces/trace?trace_id=trace_094ade8170f9482a92af5991fb0d9fdd


View Traces [here](https://platform.openai.com/traces?group_id=Dummy+Group)

# Add Viz

requires GraphViz extension to view

In [ ]:
from agents.extensions.visualization import draw_graph

try:
    draw_graph(agent, filename="agent_graph.gv")
except Exception as e:
    print(f"An error occurred: {e}")

An error occurred: failed to execute WindowsPath('dot'), make sure the Graphviz executables are on your systems' PATH
